In [1]:
import csv
import gzip
import os
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import torch

import scvi
from scvi.dataset import GeneExpressionDataset, CellMeasurement, AnnDatasetFromAnnData, Dataset10X
from scvi.models import VAE, TOTALVI
from scvi.inference import TotalPosterior, TotalTrainer, Posterior, UnsupervisedTrainer

import anndata
import scanpy as sc
import umap


save_path = "../../data/10X"
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
dataset = Dataset10X(
    dataset_name="293t",
    save_path=save_path,
    measurement_names_column=1,
    dense=True,
)

[2020-01-31 10:09:49,408] INFO - scvi.dataset.dataset | Downloading file at /Users/adamgayoso/Google Drive/Berkeley/Software/scVI/data/10X/293t/filtered_gene_bc_matrices.tar.gz
[2020-01-31 10:09:50,402] INFO - scvi.dataset.dataset10X | Preprocessing dataset
[2020-01-31 10:09:50,403] INFO - scvi.dataset.dataset10X | Extracting tar file
[2020-01-31 10:10:08,377] INFO - scvi.dataset.dataset10X | Finished preprocessing dataset
[2020-01-31 10:10:08,957] WARNING - scvi.dataset.dataset | Gene names are not unique.
[2020-01-31 10:10:08,958] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-01-31 10:10:08,959] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-01-31 10:10:10,204] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-01-31 10:10:10,508] INFO - scvi.dataset.dataset | Downsampled from 2885 to 2885 cells


In [5]:
gene_means = dataset.X.mean(0)
lib_size = np.random.uniform(0.9, 1.1, size=(5000,1))
new_X_mean = lib_size * gene_means

In [9]:
expressed_genes = np.mean(dataset.X > 0, axis=0) > 0.1

In [10]:
new_X = np.random.poisson(new_X_mean)

In [11]:
new_X = new_X[:, expressed_genes]

In [12]:
new_X

array([[ 1,  1,  6, ...,  6,  0, 43],
       [ 2,  3,  7, ...,  5,  3, 42],
       [ 0,  1,  6, ...,  7,  1, 42],
       ...,
       [ 0,  5,  8, ..., 10,  2, 44],
       [ 0,  0, 10, ...,  4,  4, 43],
       [ 0,  1,  5, ...,  2,  1, 41]])

In [14]:
new_data = GeneExpressionDataset()
new_data.populate_from_data(new_X)

[2020-01-31 10:14:59,655] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-01-31 10:14:59,656] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


In [20]:
scvi_vae = VAE(new_data.nb_genes, reconstruction_loss="nb")
scvi_trainer = UnsupervisedTrainer(
    scvi_vae,
    new_data,
    train_size=0.90,
    test_size=0.10,
    use_cuda=False,
    frequency=10
)

In [21]:
scvi_trainer.train(lr=1e-3, n_epochs=200)

[2020-01-31 10:18:03,289] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.


KeyboardInterrupt: 

In [ ]:
full_scvi_posterior = trainer.create_posterior(
    scvi_vae, dataset, indices=np.arange(len(dataset)),
)
latent, _, _ = full_scvi_posterior.get_latent()